## Scopes and Namespaces

In [14]:
def scope_test():
    def do_local():
        spam = "local spam"
    def do_nonlocal():
        nonlocal spam
        spam = 'non local spam'
    def do_global():
        global spam
        spam = 'global spam'
    
    spam = 'test spam'
    do_local()
    print("After local assignment:",spam)
    do_nonlocal()
    print('After nonlocal assigment:',spam)
    do_global()
    print('After global assignment:',spam)

scope_test()
print('In global scope:',spam)
    

After local assignment: test spam
After nonlocal assigment: non local spam
After global assignment: non local spam
In global scope: global spam


See how, the 
- local assignment inside the function do_local, doesnot change the spam variable.
- non local assigment inside the non_local function changes the spam variable outside it's local scope, into scope_test's binding of spam
- global assigment changes the variable globally in the module-level binding

### Class Objects

Class objects support two kinds of operations: attribute references and instantiation.

Valid attribute refrences are all the names that were in the class's namespace when the class was created.

In [15]:
class MyClass:
    """An example class"""
    i =1223
    def f(self):
        return 'Hey, Man!'

MyClass.i and MyClass.f are valid attributes.  
MyClass.\__doc\__ is also a valid attribute returning docstring of that class.

When a class defines an \__init\__() method, class instantiation automatically invokes \__init\__() for the newly-created class instance. 
``` 
    def __init__(self):
        self.data = []
```


In [38]:
class MyClass:
    """An example class"""
    i =1223
    
    def __init__(self,i):
        self.i = i
    def f(self):
        return 'Hey, Man!'
    

In [42]:
# Empty class instantiation
x=MyClass

In [43]:
x.i

1223

In [44]:
# Class instantiation using contructers
x = MyClass(4)

In [47]:
x.i

4

### Class and instance variables

In [50]:
class Dog:
    kind = 'canine'   # class variable shared by all instances
    
    def __init__(self,name):
        self.name = name   # instance variable unique to each instance

d = Dog('Fido')
e = Dog('Bruno')

In [52]:
d.kind,e.kind   # shared by all dogs

('canine', 'canine')

In [53]:
d.name,e.name   # unique to each dog

('Fido', 'Bruno')

Methods may call other methods by using method attribute of self argument.

In [54]:
class Inventory:
    def __init__(self):
        self.data = []         # instance variable, 'data'
    
    def add(self,x):           # method, 'add'
        self.data.append(x)    # access object variable
    
    def add_twice(self,x):     # method, 'add_twice'
        self.add(x)            # call class method, 'add'
        self.add(x)

In [57]:
i = Inventory()
i.add('nail_cutter')
i.add_twice('polish')
i.data

['nail_cutter', 'polish', 'polish']

In [58]:
i2 = Inventory()
i2.add_twice('dish_washer')

In [59]:
i2.data

['dish_washer', 'dish_washer']

Each value is an object, and therefore has a class (also called its type). It is stored as object.\__class\__.

In [62]:
i2.__class__

__main__.Inventory

## Inheritance

Derived classes may override methods of their base classes.  
A simple way to call the base class method directly: just call BaseClassName.methodname(self, arguments).   

Python has two built-in functions that work with inheritance:

- Use __isinstance()__ to check an instance’s type: isinstance(obj, int) will be True only if obj.\__class\__ is int or some class derived from int.

- Use __issubclass()__ to check class inheritance: issubclass(bool, int) is True since bool is a subclass of int. However, issubclass(float, int) is False since float is not a subclass of int.

### Multiple inheritance
```class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .
    .
    .
    <statement-N>
```  
For most purposes, in the simplest cases, you can think of the search for attributes inherited from a parent class as __depth-first, left-to-right__, not searching twice in the same class where there is an overlap in the hierarchy. Thus, if an attribute is not found in DerivedClassName, it is searched for in Base1, then (recursively) in the base classes of Base1, and if it was not found there, it was searched for in Base2, and so on.



### Private Variables
“Private” instance variables that cannot be accessed except from inside an object don’t exist in Python.   
However, there is a convention that is followed by most Python code: a name prefixed with an underscore (e.g. _spam) should be treated as a non-public part of the API (whether it is a function, a method or a data member).   

It should be considered an implementation detail and subject to change without notice.


In [64]:
class Mapping:
    def __init__(self,iterable):
        self.items_list = []
        self.__update(iterable)
    
    def update(self,iterable):
        for item in iterable:
            self.items_list.append(item)
    
    __update = update   # private copy  of the orginal update() method

class MappingSubClass(Mapping):
    
    def update(self,keys,values):
        # provides new signature for update()
        # but does not break __init__() 
        for item in zip(keys,values):
            self.items_list.append(item)
            


In [66]:
m = Mapping

### Iterators

In [73]:
# As most container objects can be looped using for loop
for element in [1,2,3]:
    print(element)
for element in (1,2,3):
    print(element)
for keys in {'one':1,'two':2}:
    print(keys)
for char in 'abc':
    print(char)
for line in open('fibo.py'):
    print(line,end='')

1
2
3
1
2
3
one
two
a
b
c
def fib(n):
	x,y = 0,1
	while y < n:
		print(y,end=' ')
		x,y = y,x+y
	print()

def fib2(n):
	result = []
	x,y = 0,1
	while y < n:
		result.append(y)
		x,y = y,x+y
	return result

if __name__ == "__main__":
    import sys
    fib2(int(sys.argv[1]))


Behind the scenes, the for statement calls iter() on the container object.   
The function returns an iterable object that defines the method \__next()\__ which access the elements in the container one at a time.  
When there are no more elements, \__next()\__ raises a StopIteration exception which tells the for loop to terminate.

In [84]:
s = 'abc'
it = iter(s)
it

In [85]:
next(it),next(it),next(it)

('a', 'b', 'c')

In [86]:
next(it)

StopIteration: 

Having seen the mechanincs of iterator, you can add iterator behavior to your classes. 
Define an \__iter\__() method that returns a \__next\__() method.  
If the class defines \__next\__() method, then \__iter\__() can return self.

In [189]:
class Reverse:
    """Iterator for looping over a sequence backwards"""
    
    def __init__(self,data):
        self.data = data
        self.index = len(data)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index ==0:
            raise StopIteration
        self.index-=1
        return self.data[self.index]

In [190]:
rev = Reverse([1,2,3,4])

In [191]:
iter(rev)

In [192]:
for char in rev:
    print(char)

4
3
2
1


### Generators

Generators are a simple and powerful tool for creating iterators.   
They are written like regular functions but use the yield statement whenever they want to return data.  
Each time next() is called on it, the generator resumes where it left off (it remembers all the data values and which statement was last executed). 

In [212]:
def reverse(data):
    for index in range(len(data)-1,-1,-1):
        yield data[index]

In [219]:
reverse([1,2,3,4])

<generator object reverse at 0x7f55e036c150>

In [217]:
for char in reverse('golf'):
    print(char)

f
l
o
g


What makes generators more compact is that the \__iter\__() and \__next\__() method are created automatically.
Also, the local variables and execution state are saved automatically between calls.
It raises the StopIteration exception on termination.

#### Generator Expressions
Generator expressions are more compact but less versatile than full generator definations, and tend to be more __memory friendly__ than list comprehensions.  

These expressions are designed for situations where the generator is used right away by an __enclosing function.__  
Expression is similar to that of list comprehension, but with parenthesis instead of brackets.

In [242]:
sum((i*i for i in range(5)))

30

In [244]:
xvect = [1,2,3,4]
yvect = [5,10,15,20]
sum((x*y for x,y in zip(xvect,yvect)))

150

In [236]:
from math import pi,sin

In [237]:
sine_table = {x:sin(x*pi/180) for x in range(0,91)}


In [238]:
unique_words = 

{0: 0.0,
 1: 0.01745240643728351,
 2: 0.03489949670250097,
 3: 0.05233595624294383,
 4: 0.0697564737441253,
 5: 0.08715574274765817,
 6: 0.10452846326765346,
 7: 0.12186934340514748,
 8: 0.13917310096006544,
 9: 0.15643446504023087,
 10: 0.17364817766693033,
 11: 0.1908089953765448,
 12: 0.20791169081775931,
 13: 0.224951054343865,
 14: 0.24192189559966773,
 15: 0.25881904510252074,
 16: 0.27563735581699916,
 17: 0.29237170472273677,
 18: 0.3090169943749474,
 19: 0.32556815445715664,
 20: 0.3420201433256687,
 21: 0.35836794954530027,
 22: 0.374606593415912,
 23: 0.3907311284892737,
 24: 0.40673664307580015,
 25: 0.42261826174069944,
 26: 0.4383711467890774,
 27: 0.45399049973954675,
 28: 0.4694715627858908,
 29: 0.48480962024633706,
 30: 0.49999999999999994,
 31: 0.5150380749100542,
 32: 0.5299192642332049,
 33: 0.5446390350150271,
 34: 0.5591929034707469,
 35: 0.573576436351046,
 36: 0.5877852522924731,
 37: 0.6018150231520483,
 38: 0.6156614753256582,
 39: 0.6293203910498374,
 40: 0.